In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
from sklearn.metrics.pairwise import pairwise_distances

import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
pd.options.display.max_colwidth = 600

In [8]:
nRowsRead = None # specify 'None' if want to read whole file

df_poems = pd.read_csv('../data/raw/data_original.csv', delimiter=',', nrows = nRowsRead)
df_poems.dataframeName = 'PoetryFoundationData.csv'
nRow, nCol = df_poems.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 13854 rows and 5 columns


In [9]:
### Data cleaning steps
## Drop unnecessary columns
df_clean = df_poems.drop(columns=['Unnamed: 0'])

## Drop NAs
# Drop NA fields
df_clean = df_clean.dropna(subset=['Tags', 'Poem'])

# Drop instances with no alpha-numeric values
drop_index = df_clean[~df_clean.Poem.str.contains("[A-Za-z0-9]")].index
df_clean = df_clean.drop(index = drop_index)

# Remove double spacing
df_clean.Poem = df_clean.Poem.str.replace("\r\r\n\r\r\n\r\r\n\r\r\n", "\r\r\n \r\r\n")

# Remove leading and trailing spaces in Poem
df_clean.Poem = df_clean.Poem.str.lstrip()
df_clean.Poem = df_clean.Poem.str.rstrip()

# Remove leading and trailing spaces in Title
df_clean.Title = df_clean.Title.str.lstrip()
df_clean.Title = df_clean.Title.str.rstrip()

## Create new general tags
df_clean["Tags_general"] = np.nan

## Reset index
df_clean = df_clean.reset_index(drop = True)

In [10]:
df_clean

,Title,Poem,Poet,Tags,Tags_general
0,Invisible Fish,"Invisible fish swim this ghost ocean now described by waves of sand, by water-worn rock. Soon the fish will learn to walk. Then humans will come ashore and paint dreams on the dying stone. Then later, much later, the ocean floor will be punctuated by Chevy trucks, carrying the dreamers’ decendants, who are going to the store.",Joy Harjo,"Living,Time & Brevity,Relationships,Family & Ancestors,Nature,Landscapes & Pastorals,Seas, Rivers, & Streams,Social Commentaries,History & Politics",NaN
1,Don’t Bother the Earth Spirit,"Don’t bother the earth spirit who lives here. She is working on a story. It is the oldest story in the world and it is delicate, changing. If she sees you watching she will invite you in for coffee, give you warm bread, and you will be obligated to stay and listen. But this is no ordinary story. You will have to endure earthquakes, lightning, the deaths of all those you love, the most blinding beauty. It’s a story so compelling you may never want to leave; this is how she traps you. See that stone finger over there? That is the only one who ever escaped.",Joy Harjo,"Religion,The Spiritual,Mythology & Folklore,Fairy-tales & Legends",NaN
2,"[""Hour in which I consider hydrangea""]","Hour in which I consider hydrangea, a salt or sand plant, varietal, the question of varietals, the diet of every mother I know, 5 pounds feels like 20, I have lost … I have lost, yes, a sense of my own possible beauty, grown external, I externalize beauty. Beauty occurs on the surface of plants; the sun darkens the skin of my child, he is so small, he is beautiful (I can see; it is obvious) and everything about him is beautiful. His hand swells from the bite [spread?] of some insect[’s] venom because he is small. He appears to feel nothing. He smashes his skull against the floor. He scream...",Simone White,"Living,Parenthood,The Body,The Mind,Nature,Trees & Flowers",NaN
3,scars,my father’s body is a map\r\r\na record of his journey\r\r\n \r\r\nhe carries a bullet\r\r\nlodged in his left thigh\r\r\nthere is a hollow where it entered\r\r\na protruding bump where it sleeps\r\r\nthe doctors say it will never awaken\r\r\n \r\r\nit is the one souvenir he insists on keeping\r\r\nmother has her own opinionsbố cùa con điên—your father is crazy\r\r\n \r\r\nas a child\r\r\ni wanted a scar just like my father’s\r\r\nbold and appalling a mushroom explosion\r\r\nthat said i too was at war\r\r\ninstead i settled for a grain of rice\r\r\na scar so small look closely there\r\...,Truong Tran,"The Body,Family & Ancestors",NaN
4,what remains two,it has long been forgotten this practice of the mother\r\r\nweaning a child she crushes the seeds of a green\r\r\nchili rubs it to her nipple what the child feels\r\r\nshe too will share in this act of love\r\r\nmy own mother says it was not meant\r\r\nto be cruel when cruelty she tells me\r\r\nis a child’s lips torn from breast as proof\r\r\nback home the women wear teeth marks,Truong Tran,"Infancy,Parenthood,The Body",NaN
...,...,...,...,...,...
12852,!,"Dear Writers, I’m compiling the first in what I hope is a series of publications I’m calling artists among artists. The theme for issue 1 is “Faggot Dinosaur.” I hope to hear from you! Thank you and best wishes.",Wendy Videlock,"Relationships,Gay, Lesbian, Queer,Arts & Sciences,Poetry & Poets,Social Commentaries,Gender & Sexuality",NaN
12853,1 January 1965,"The Wise Men will unlearn your name.\r\r\nAbove your head no star will flame.\r\r\nOne weary sound will be the same—\r\r\nthe hoarse roar of the gale.\r\r\nThe shadows fall from your tired eyes\r\r\nas your lone bedside candle dies,\r\r\nfor here the calendar breeds nights\r\r\ntill stores of candles fail.\r\r\nWhat prompts this melancholy key?\r\r\nA long familiar melody.\r\r\nIt sounds again. So let it be.\r\r\nLet it sound from this night.\r\r\nLet it sound in my hour of death—\r\r\nas gratefulness of eyes and lips\r\r\nfor that which sometimes makes us

In [13]:
# Initialize dictionary, list, and initial index
tag_idx = {}
corpus = []
idx = 0

# For loop for tokenization
for i in range(len(df_clean)):    
    tag = df_clean['Tags'][i]
    tag_lower = tag.lower()
    tokens = tag_lower.split(',')
    corpus.append(tokens)
    for tag in tokens:
        if tag not in tag_idx:
            tag_idx[tag] = idx
            idx += 1

In [14]:
# Get the number of items and tokens 
M = len(df_clean)
N = len(tag_idx)

# Initialize a matrix of zeros
df_clean_mat = np.zeros((M, N))

In [15]:
# Define the oh_encoder function
def oh_encoder(tokens):
    x = np.zeros(N)
    for tag in tokens:
        # Get the index for each tag
        idx = tag_idx[tag]
        # Put 1 at the corresponding indices
        x[idx] = 1
    return x

In [16]:
# Make a document-term matrix
i = 0
for tokens in corpus:
    df_clean_mat[i, :] = oh_encoder(tokens)
    i+=1

In [17]:
df_clean_mat.shape

(12857, 129)

In [18]:
print(df_clean_mat[0])

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [19]:
print(df_clean_mat[1])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [20]:
df_clean_mat_dist = 1 - pairwise_distances(df_clean_mat, metric = "jaccard")

c:\users\aznor\anaconda3\envs\poem\lib\site-packages\sklearn\metrics\pairwise.py:1735: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [21]:
print(df_clean_mat_dist)

[[1.         0.         0.13333333 ... 0.16666667 0.         0.11764706]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.13333333 0.         1.         ... 0.125      0.         0.        ]
 ...
 [0.16666667 0.         0.125      ... 1.         0.         0.1       ]
 [0.         0.         0.         ... 0.         1.         0.11111111]
 [0.11764706 0.         0.         ... 0.1        0.11111111 1.        ]]


In [22]:
# Import k-means to perform clusters
from sklearn.cluster import KMeans

# Create a KMeans object with k clusters and save as km
km = KMeans(n_clusters=80)

# Fit the k-means object with distance matrix
km.fit(df_clean_mat_dist)

clusters = km.labels_.tolist()

In [23]:
# Create a column cluster to denote the generated cluster for each movie
df_clean["cluster"] = clusters

In [24]:
# Display number of films per cluster (clusters from 0 to 4)
df_clean['cluster'].value_counts().head(20)

35    338
29    316
40    313
27    271
3     266
53    265
32    256
30    253
23    243
51    228
78    226
14    215
43    211
12    207
13    207
16    204
38    202
26    199
74    199
33    197
Name: cluster, dtype: int64

In [25]:
#46    350 # Life, love, relationships
#26    345 # Love
#69    332 # Love, complicated, heartache
#20    301 # Life, living
#63    272 # Life, coming of age
#58    272 # Arts, sci, phil
#28    272 # Life, social commentary, sociology
#36    269 # Nature
#38    268 # Living, family social relationships, social commentary
#21    247 # Religion
#49    245 # Humans and nature
#5     241 # Life, growing old
#41    238 # Life and philo
#8     229 # Nature
#62    209 # Culture
#12    207 # Humans and nature
#79    203 # Life, journey, youth and learning,
#52    202 # Outdoor, activities
#13    201 # Life, relationship and family
#73    199 # Life, health, heartache and loss

'''
1) Love 
2) Relationships and family
3) Social commentary and philosophy
4) Nature and animals
5) Life and coming of age
'''

'\n1) Love \n2) Relationships and family\n3) Social commentary and philosophy\n4) Nature and animals\n5) Life and coming of age\n'

In [26]:
## Select the cluster descriptions
cluster_love_fam = [46, 26, 69, 38, 13]
cluster_soc_phi_cul = [58, 28, 21, 41, 62]
cluster_nature = [36, 49, 12, 52]
cluster_life_death = [5, 79, 73, 20, 63]

In [27]:
custer_defined = cluster_love_fam + cluster_soc_phi_cul + cluster_nature + cluster_life_death

In [28]:
df_clean[~df_clean.cluster.isin(custer_defined)]

,Title,Poem,Poet,Tags,Tags_general,cluster
0,Invisible Fish,"Invisible fish swim this ghost ocean now described by waves of sand, by water-worn rock. Soon the fish will learn to walk. Then humans will come ashore and paint dreams on the dying stone. Then later, much later, the ocean floor will be punctuated by Chevy trucks, carrying the dreamers’ decendants, who are going to the store.",Joy Harjo,"Living,Time & Brevity,Relationships,Family & Ancestors,Nature,Landscapes & Pastorals,Seas, Rivers, & Streams,Social Commentaries,History & Politics",NaN,17
1,Don’t Bother the Earth Spirit,"Don’t bother the earth spirit who lives here. She is working on a story. It is the oldest story in the world and it is delicate, changing. If she sees you watching she will invite you in for coffee, give you warm bread, and you will be obligated to stay and listen. But this is no ordinary story. You will have to endure earthquakes, lightning, the deaths of all those you love, the most blinding beauty. It’s a story so compelling you may never want to leave; this is how she traps you. See that stone finger over there? That is the only one who ever escaped.",Joy Harjo,"Religion,The Spiritual,Mythology & Folklore,Fairy-tales & Legends",NaN,30
2,"[""Hour in which I consider hydrangea""]","Hour in which I consider hydrangea, a salt or sand plant, varietal, the question of varietals, the diet of every mother I know, 5 pounds feels like 20, I have lost … I have lost, yes, a sense of my own possible beauty, grown external, I externalize beauty. Beauty occurs on the surface of plants; the sun darkens the skin of my child, he is so small, he is beautiful (I can see; it is obvious) and everything about him is beautiful. His hand swells from the bite [spread?] of some insect[’s] venom because he is small. He appears to feel nothing. He smashes his skull against the floor. He scream...",Simone White,"Living,Parenthood,The Body,The Mind,Nature,Trees & Flowers",NaN,57
3,scars,my father’s body is a map\r\r\na record of his journey\r\r\n \r\r\nhe carries a bullet\r\r\nlodged in his left thigh\r\r\nthere is a hollow where it entered\r\r\na protruding bump where it sleeps\r\r\nthe doctors say it will never awaken\r\r\n \r\r\nit is the one souvenir he insists on keeping\r\r\nmother has her own opinionsbố cùa con điên—your father is crazy\r\r\n \r\r\nas a child\r\r\ni wanted a scar just like my father’s\r\r\nbold and appalling a mushroom explosion\r\r\nthat said i too was at war\r\r\ninstead i settled for a grain of rice\r\r\na scar so small look closely there\r\...,Truong Tran,"The Body,Family & Ancestors",NaN,35
4,what remains two,it has long been forgotten this practice of the mother\r\r\nweaning a child she crushes the seeds of a green\r\r\nchili rubs it to her nipple what the child feels\r\r\nshe too will share in this act of love\r\r\nmy own mother says it was not meant\r\r\nto be cruel when cruelty she tells me\r\r\nis a child’s lips torn from breast as proof\r\r\nback home the women wear teeth marks,Truong Tran,"Infancy,Parenthood,The Body",NaN,35
...,...,...,...,...,...,...
12849,The Bean Eaters,"They eat beans mostly, this old yellow pair. \r\r\nDinner is a casual affair.\r\r\nPlain chipware on a plain and creaking wood, \r\r\nTin flatware.\r\r\nTwo who are Mostly Good.\r\r\nTwo who have lived their day,\r\r\nBut keep on putting on their clothes \r\r\nAnd putting things away.\r\r\nAnd remembering ...\r\r\nRemembering, with twinklings and twinges,\r\r\nAs they lean over the beans in their rented back room that is full of beads and receipts and dolls and cloths, tobacco crumbs, vases and fringes.",Gwendolyn Brooks,"Living,Growing Old,Marriage & Companionship,Relationships,Home Life",NaN,66
12852,!,"Dear Writers, I’m compiling the first in what I hope is a series of publications I’m calling artists among artists. The theme for issue 1 is “Faggot Dinosaur.” I hope to hear from you! Thank you and best wishes.",Wendy Videlock,"Relationships,Gay, Lesbian, Queer,Arts 

In [29]:
## Assign clusters to data and export
df_clean_love_fam = df_clean[df_clean.cluster.isin(cluster_love_fam)]
df_clean_soc_phi_cul = df_clean[df_clean.cluster.isin(cluster_soc_phi_cul)]
df_clean_nature = df_clean[df_clean.cluster.isin(cluster_nature)]
df_clean_life_death = df_clean[df_clean.cluster.isin(cluster_life_death)]
df_clean_others = df_clean[~df_clean.cluster.isin(custer_defined)]

## **Notes for EDA**
* For Poems field, 2016 and 101 containing no alpha numeric values

* For tags field, there are missing values (955 or 6.9%), and over 8,297 distinct categories. Largest category is at 1.3% of all data. * Suggest to drop the missing values (if current tags are important), and re-categorise the data to broader topics using some ML technique. Not possible to manually tag because n = 200 categories means only 3k+ data will be used*

* For poet field, there are over 3,128 unique poets. The largest number of poems written is 85 poems or 2.7% of all data. May want to group the poet by other categories?
 
* Drop field called "Unnamed: 0"




## **Data cleaning steps**
* Drop field called "Unnamed: 0"

* For Poems field, 2016 and 101 containing no alpha numeric values

* For tags field, there are missing values (955 or 6.9%), and over 8,297 distinct categories. Largest category is at 1.3% of all data. * Suggest to drop the missing values (if current tags are important), and re-categorise the data to broader topics using some ML technique. Not possible to manually tag because n = 200 categories means only 3k+ data will be used*

* For poet field, there are over 3,128 unique poets. The largest number of poems written is 85 poems or 2.7% of all data. May want to group the poet by other categories?

In [ ]:
df_poems.describe()

In [ ]:
df_poems.Tags.nunique()

In [ ]:
85 / 3128 * 100

In [ ]:
cv_tags = df_poems.Tags.value_counts(dropna = False)

In [ ]:
largest_tags = cv_tags.head(200).index

In [ ]:
df_poems[df_poems.Tags.isin(largest_tags)].count()

In [ ]:
df_poems.Tags.value_counts(dropna = False).head(20)

In [ ]:
df_poems.sort_values('Poem').Poem.str.contains("\r\r\n\r\r\n").sum()

In [ ]:
df_poems.sort_values('Poem')

In [ ]:
df_poems[~df_poems.Poem.str.contains("[A-Za-z0-9]")]

In [ ]:
def _missing_values_table(df):
    '''
    This function takes a pandas dataframe input and shows the number and proportion of null fields.

    source: https://stackoverflow.com/questions/26266362/how-to-count-the-nan-values-in-a-column-in-pandas-dataframe/39734251#39734251

    '''
    
    # Total missing values
    mis_val = df.isnull().sum()
        
    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)
        
    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
        
    # Print some summary information
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
        " columns that have missing values.")
        
    # Return the dataframe with missing information
    return mis_val_table_ren_columns

In [ ]:
_missing_values_table(df_clean)